# Section 0: Importing libraries

In [39]:
import pandas as pd

# Section 1: Data pre-processing
---
1.1 [Loading the data](#loading)

1.2 [Dataset properties](#properties)

### Loading the data <a id="loading"></a>

In [40]:
# main data path and relative paths to the datasets
DATA_PATH = "./data/"
DATASET_PATH = {
    "movie_metadata": DATA_PATH + "movie.metadata.tsv",
    "character_metadata": DATA_PATH + "character.metadata.tsv",
    "plot_summaries": DATA_PATH + "plot_summaries.txt",
}

In [41]:
# columns used when loading a specific dataset (for the attribute "names" in pandas.read_csv)
DATASET_COLUMNS = {
    "movie_metadata": ["wikipedia_id", "freebase_movie_id", "name", "release_date", "revenue", "runtime", "languages", "countries", "genres"],
    "character_metadata": ["wikipedia_id", "freebase_movie_id", "release_date", "character_name", "birthday", "gender", "height", "ethnicity", "name", "age_at_release", "freebase_map", "freebase_char_id", "freebase_actor_id"]
}

In [42]:
# load the datasets as pandas dataframes
movie_metadata = pd.read_csv(DATASET_PATH["movie_metadata"], sep='\t', names=DATASET_COLUMNS["movie_metadata"], index_col=False, header=None)
character_metadata = pd.read_csv(DATASET_PATH["character_metadata"], sep='\t', names=DATASET_COLUMNS["character_metadata"], index_col=False, header=None)

### 1.2 Dataset properties <a id="properties"></a>

In [43]:
print("Number of movie rows: ", movie_metadata.shape[0])
print("Number of features per movie: ", movie_metadata.shape[1])
# a random movie sample
movie_metadata.sample(1)

Number of movie rows:  81741
Number of features per movie:  9


,wikipedia_id,freebase_movie_id,name,release_date,revenue,runtime,languages,countries,genres
3298,27329170,/m/0by00vv,Tōki Rakujitsu,1992-07-04,NaN,119.0,{},"{""/m/03_3d"": ""Japan""}","{""/m/0gw5n2f"": ""Japanese Movies""}"


In [44]:
print("Number of character rows: ", character_metadata.shape[0])
print("Number of features per character: ", character_metadata.shape[1])
# a random character sample
character_metadata.sample(1)

Number of character rows:  450669
Number of features per character:  13


,wikipedia_id,freebase_movie_id,release_date,character_name,birthday,gender,height,ethnicity,name,age_at_release,freebase_map,freebase_char_id,freebase_actor_id
53384,20935471,/m/05b2d7t,1988-10-27,Natalie,NaN,F,NaN,NaN,Saskia Schlicht,NaN,/m/0n50p17,/m/0n537wh,/m/0n50p1b
